### 1. Update environment

In [1]:
!pip install --upgrade boto3 
!pip install --upgrade sagemaker 
!pip install nvidia-pyindex && pip install nvidia-cublas
!sudo yum update -y
!sudo yum install git-lfs git -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 52.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.33.4
    Uninstalling botocore-1.33.4:
      Successfully uninstalled botocore-1.33.4
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.8.2
    Uninstalling s3transfer-0.8.2:
      Successfully uninstalled s3transfer-0.8.2
  Attempting uninstall: boto3
    Found existing installation: boto3 1.33.4
    Uninstalling boto3-1.33.4:
      Successfully uninstalled boto3-1.33.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.31.4 requires botocore==1.33.4, but you have botocore 1.34.0 which is incompatible.


In [2]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
# Select 7b or 13b model parameter
repo = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"
#repo = "https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_0.gguf"
#repo ="https://huggingface.co/TheBloke/blossom-v3-baichuan2-7B-GGUF/resolve/main/blossom-v3-baichuan2-7b.Q5_0.gguf"
model_id="llm_gguf"
s3_model_prefix =model_id
s3_code_prefix=model_id
bucket = "sagemaker-"+region+"-" + account_id  #specify your s3 dir to store model repo example-> s3://your-bucket-name/your-s3-dir
s3 = boto3.client('s3')
directory_name = model_id  #it's name of your folders
s3.put_object(Bucket=bucket, Key=(directory_name+'/'))



{'ResponseMetadata': {'RequestId': 'SCYB5N4TCXZN4QSP',
  'HostId': 'c+gaR9vSeNUoqXNXZzPdb2MjicWd6QX4XvIeqc4j4VYwSqX1xpVIMVWEKta8dWjoMsfARaXu7+0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'c+gaR9vSeNUoqXNXZzPdb2MjicWd6QX4XvIeqc4j4VYwSqX1xpVIMVWEKta8dWjoMsfARaXu7+0=',
   'x-amz-request-id': 'SCYB5N4TCXZN4QSP',
   'date': 'Thu, 14 Dec 2023 10:41:54 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
 'ServerSideEncryption': 'AES256'}

In [4]:
!mkdir $model_id
s3_location = "s3://"+bucket + "/" + model_id + "/"
local_model_dir = "./" + model_id 

!echo $s3_location
!echo $repo
!echo $local_model_dir
!echo $model_id

s3://sagemaker-us-east-1-524648928792/llm_gguf/
https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf
./llm_gguf
llm_gguf


In [5]:
!wget $repo

--2023-12-14 10:41:54--  https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf
Resolving huggingface.co (huggingface.co)... 13.32.151.42, 13.32.151.126, 13.32.151.56, ...
Connecting to huggingface.co (huggingface.co)|13.32.151.42|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/8d/b1/8db1d1f73b4caa58e947ccbfe2fb27ac5e495c2ad8457ad299d15987aee3b520/eda2a15d532bea4ce6fc14d15c2b72638243396816252f73a94dceeb0429112f?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-13b-chat.Q4_0.gguf%3B+filename%3D%22llama-2-13b-chat.Q4_0.gguf%22%3B&Expires=1702809714&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMjgwOTcxNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy84ZC9iMS84ZGIxZDFmNzNiNGNhYTU4ZTk0N2NjYmZlMmZiMjdhYzVlNDk1YzJhZDg0NTdhZDI5OWQxNTk4N2FlZTNiNTIwL2VkYTJhMTVkNTMyYmVhNGNlNmZjMTRkMTVjMmI3MjYzODI0MzM5N

In [6]:
!cd $local_model_dir && rm -rf ".ipynb_checkpoints"
!touch $local_model_dir/model
!tar czvf model.tar.gz $local_model_dir/*

./llm_gguf/model


In [7]:
s3_model = sess.upload_data("model.tar.gz", bucket, directory_name)
#!aws s3 sync $local_model_dir $s3_location

### 2. Make docker image
##### Notice: you can modify Dockfile to ADD different model weights.
##### modify 'Select Model' part with downloaded gguf file from line 38 in below cell

In [8]:
%%writefile ./Dockerfile

ARG CUDA_IMAGE="12.1.1-devel-ubuntu22.04"
FROM nvidia/cuda:${CUDA_IMAGE} as build

ENV CUDA_DOCKER_ARCH=all
ENV LLAMA_CUBLAS=1

ARG DEBIAN_FRONTEND=noninteractive

# Install the package
RUN apt-get update && \
    apt-get install -y build-essential python3 python3-pip git gcc cmake

RUN python3 -m pip install --upgrade pip pytest cmake scikit-build setuptools fastapi uvicorn sse-starlette pydantic-settings

# ADD Llama.cpp
ARG FUNCTION_DIR="/opt/program"
ENV FUNCTION_DIR="/opt/program"
WORKDIR ${FUNCTION_DIR}
RUN git clone https://github.com/ggerganov/llama.cpp.git ${FUNCTION_DIR}/llama.cpp

# Build Llama.cpp
WORKDIR ${FUNCTION_DIR}
RUN mkdir -p ${FUNCTION_DIR}/llama.cpp/build
RUN cd llama.cpp/build && cmake .. -DLLAMA_CUBLAS=ON && cmake --build . --config Release

#ADD Python file
WORKDIR ${FUNCTION_DIR}
ADD requirements.txt ${FUNCTION_DIR}
RUN pip3 install -r ${FUNCTION_DIR}/requirements.txt
RUN pip3 install flask gevent requests
ADD sm_server_api.py ${FUNCTION_DIR}

#ADD model
RUN mkdir -p ${FUNCTION_DIR}/model

# ----- Select Model-----
# ADD baichuan2-7b-chat.Q4_0.gguf ${FUNCTION_DIR}/model
# ENV MODEL_PATH="${FUNCTION_DIR}/model/baichuan2-7b-chat.Q4_0.gguf"

ADD llama-2-13b-chat.Q4_0.gguf ${FUNCTION_DIR}/model
ENV MODEL_PATH="${FUNCTION_DIR}/model/llama-2-13b-chat.Q4_0.gguf"

# ---------------------------------

ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PYTHONIOENCODING="utf8"

# Start Server
# LLama2 Context Length <= 2048, Baichuan2 Context Length <=4096, based on your model
ENV CONTEXT_LENGTH=4096
WORKDIR ${FUNCTION_DIR}
ADD docker-entrypoint.sh ${FUNCTION_DIR}
RUN chmod +x docker-entrypoint.sh

ENTRYPOINT ["./docker-entrypoint.sh"]

Writing ./Dockerfile


In [9]:
!docker build -t llm-gguf-sagemaker .

Sending build context to Docker daemon  7.366GB
Step 1/30 : ARG CUDA_IMAGE="12.1.1-devel-ubuntu22.04"
Step 2/30 : FROM nvidia/cuda:${CUDA_IMAGE} as build
12.1.1-devel-ubuntu22.04: Pulling from nvidia/cuda

8493d397: Pulling fs layer 
39a04761: Pulling fs layer 
cc89b769: Pulling fs layer 
d9024b9c: Pulling fs layer 
8a31fa53: Pulling fs layer 
8a8a6ebc: Pulling fs layer 
afc7a3ac: Pulling fs layer 
762ffdd9: Pulling fs layer 
e6fadd42: Pulling fs layer 
0fb08ae2: Pulling fs layer 
a6bba88f: Pull complete .37kB/88.37kBBExtracting  240.6MB/1.329GBExtracting  874.6MB/1.329GBDigest: sha256:7012e535a47883527d402da998384c30b936140c05e2537158c80b8143ee7425
Status: Downloaded newer image for nvidia/cuda:12.1.1-devel-ubuntu22.04
 ---> 5ed6afba2273
Step 3/30 : ENV CUDA_DOCKER_ARCH=all
 ---> Running in 26e590ae6211
Removing intermediate container 26e590ae6211
 ---> e018b13d1490
Step 4/30 : ENV LLAMA_CUBLAS=1
 ---> Running in 62b895fc534f
Removing intermediate container 62b895fc534f
 ---> 44f01590

In [10]:
#AWS ECR Login,
!docker login -u AWS -p $(aws ecr get-login-password --region $region) https://{account_id}.dkr.ecr.{region}.amazonaws.com

#Create ECR Repo
!aws ecr create-repository --repository-name llm-gguf-sagemaker --image-scanning-configuration scanOnPush=true --image-tag-mutability MUTABLE

#Tag Image
!docker tag llm-gguf-sagemaker:latest {account_id}.dkr.ecr.{region}.amazonaws.com/llm-gguf-sagemaker:latest

#Push Image to ECR
!docker push {account_id}.dkr.ecr.{region}.amazonaws.com/llm-gguf-sagemaker:latest

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:524648928792:repository/llm-gguf-sagemaker",
        "registryId": "524648928792",
        "repositoryName": "llm-gguf-sagemaker",
        "repositoryUri": "524648928792.dkr.ecr.us-east-1.amazonaws.com/llm-gguf-sagemaker",
        "createdAt": 1702551352.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": true
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}
The push refers to repository [524648928792.dkr.ecr.us-east-1.amazonaws.com/llm-gguf-sagemaker]

ada51972: Preparing 
de0c6d60: Preparing 
f

### 3. Deploy Sagemaker Endpoint(Execute this python code snippet in juypter notebook or ec2)


In [34]:
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.session import Session
import boto3

#ECR URI
image_uri = account_id+'.dkr.ecr.'+region+'.amazonaws.com/llm-gguf-sagemaker'
# This can be dummy model file
model_dir = s3_model
endpoint_name = 'pytorch-inference-llm-v1'
if "cn-" in region:
    instance_type='ml.g4dn.2xlarge'
else:
    instance_type='ml.g5.2xlarge'
# Create the SageMaker model instance
model = Model(
    image_uri=image_uri,
    role=sagemaker.get_execution_role(),
    model_data=model_dir
)


client = boto3.client('sagemaker')
try:
    response = client.delete_endpoint_config(EndpointConfigName=endpoint_name)
except:
    pass

model.deploy(
    instance_type=instance_type,
    initial_instance_count=1,
    endpoint_name = endpoint_name,
)

-------------!

#### Invoke Sagemaekr Endpoint


In [35]:
import boto3
import time

# data = {"prompt": "how to learn english?",
#            "max_tokens" :"500",
# }
data = {"prompt": "how to learn english?",
}
runtime_sagemaker_client = boto3.client(service_name="sagemaker-runtime")
#Please write your endpoint name which you want to use
endpoint_name = 'pytorch-inference-llm-v1'

body = json.dumps(data)

start = time.time()
response = runtime_sagemaker_client.invoke_endpoint(
    EndpointName = endpoint_name,
    ContentType  = "application/json",
    Body= body)

cost = time.time() - start     
result = response['Body'].read().decode('utf-8')

print('Response: ', result)
print("\nCost Time:  %s seconds" % (cost))
print('Output Chars :', len(result[12:-3]))
print('Speed: {:.2f} Chars/s'.format(len(result)/float(cost)))

Response:  {"answer":"\nThe best way to learn English is by being immersed in the language and practicing consistently. Here are some tips on how to learn English effectively:\n\n1. Set goals: Setting specific, measurable, achievable, relevant, and time-bound (SMART) goals can help you stay motivated and focused. Write down your goals and review them regularly.\n2. Practice consistently: Make a study plan and stick to it. Consistency is key when it comes to language learning. Set aside time each day or week to practice speaking, writing, and listening to English.\n3. Find a language partner: Finding someone to practice speaking with can help you improve your speaking skills and gain confidence in using the language. You can find language partners online or in person.\n4. Use authentic materials: Use real-life materials like news articles, videos, podcasts, and songs to learn new vocabulary and grammar. This will help you learn how to use English in context.\n5. Learn grammar rules: Und

#### 4.Clean Endpoint and config

In [36]:
!aws sagemaker delete-endpoint --endpoint-name $endpoint_name
!aws sagemaker delete-endpoint-config --endpoint-config-name $endpoint_name


#### Additional : Speed comparison with JumpStart endpoint with Llama2 7B Chat，for the other model deployed by JumpStart

In [ ]:
# import boto3
# import time

# payload = {
#     "inputs": [[{"role": "user", "content": "how to learn english?"}]], 
#     "parameters": {"max_new_tokens": 1000, "top_p": 0.9, "temperature": 0.6}
# }
# endpoint_name = 'pytorch-inference-llm-v1'

# client = boto3.client("sagemaker-runtime")
# start = time.time()
# response = client.invoke_endpoint(
#     EndpointName=endpoint_name,
#     ContentType="application/json",
#     Body=json.dumps(payload),
#     CustomAttributes="accept_eula=true",
# )
# cost = time.time() - start     

# response = response["Body"].read().decode("utf8")
# print('Response: ', response[69:-11])
# print("Cost Time:  %s seconds" % (cost))
# print('Output Chars :', len(response[69:-11]))
# print('Speed: {:.2f} Chars/s'.format(len(response)/float(cost)))